In [ ]:
import json
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
results_root = Path(f"../results/generation/mnist/ae_tryout/")

results_root = Path(f"../results/generation/mnist/wae_40_40_v5/")

# results_root = Path(f"../results/generation/svhn/wae_v9/")
# results_root = Path(f"../results/generation/svhn/ae_v3")
# results_root = Path(f"../results/generation/svhn/wae_v10_long_training/")

# results_root = Path(f"../results/generation/svhn/ae_v3_long_training/")



results_root = Path(f"../results/generation/celeba/64x64/experiments_v5_long_training/")


In [ ]:
exp_results = dict()

for p in results_root.glob("*/history.json"):
    with p.open("r") as f:
        exp_results[p.parent.name] = json.load(f)
        
exp_results.keys()

In [ ]:
metrics_names = list(exp_results.values())[0][0]["metrics"].keys()
set(metrics_names)

In [ ]:
for metric in metrics_names:    
    plt.figure(figsize=(10, 5))
    for exp_name, results in exp_results.items():
        if metric not in results[0]["metrics"]:
            continue
        plt.plot(
            [r["epoch"] for r in results], 
            [r["metrics"][metric]["val"] for r in results], 
            label=f"{exp_name}_val"
        )
        
#         plt.plot(
#             [r["epoch"] for r in results], 
#             [r["metrics"][metric]["train"] for r in results], 
#             label=f"{exp_name}_train"
#         )

    plt.title(metric)
    plt.legend()
    plt.show()

In [ ]:
df = pd.DataFrame([
    {
        "experiment_name": e_name,
        **{
            f"final_{m_name}": r[-1]["metrics"][m_name]["val"]
            for m_name in metrics_names
        }
    }
    for (e_name, r)
    in exp_results.items()
    if "old" not in e_name
]).sort_values(
    "final_fid_sampling" if "wae" in results_root.name else "final_ssim", 
    ascending=False )

df

In [ ]:
ds = results_root.relative_to("../results/generation/").parts[0]
exp_name = f"metrics_generation_{ds}_{results_root.name}"

print(exp_name)

df.to_latex(f"{exp_name}.tex", index=False)